# Sprint 3 - DEMO
## Team Network Training

# 1 Data ja datan esikäsittely

## 1.1 Datan muoto

### ObjectManagerista karsittu versio json-muodossa

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_12b67741c6aabadb5e10bcf2f4381612.png)

## 1.2 Esikäsittelyn ensimmäinen idea - jako yhdeksään blokkiin

Jako yhdeksään blokkiin toteutettiin määrittämällä arkin minimi- ja maksimikoordinaatit. Näiden perusteella laskettiin tasavälisetkohdat, joista arkki viipaloitiin yhtä suuriin alueisiin. Toistaseksi emme ole vielä miettineet, mitä tehdä pinta-alaltaan erikokoisten arkkien kanssa (niitä ei varmistetuissa pareissa ole, vaikka koko +300 arkin setissä onkin).

Aluksi luulimme, että joudumme itse laskemaan minimi- ja maksimikoordinaatit. Myöhemmin osoittautui, että karsitusta objektidatasta oli vain jäänyt nurkkakoordinaatit pois. Kuitenkin tämä yritys määrittää koordinaatit itse - ja tarkoituksella ensisijaisesti nimenomaan pelkästään objektidataa käyttämällä, siis lukematta suoraan kuvan pikseleitä - osoittautui erittäin hyödylliseksi.

Tutkimustyön myötä huomasimme vähän vahingossa muita kummallisuuksia datassa sekä yhden tekemäämme blokkeihin jakoon liittyvän ongelman.

![image](https://gitlab.dclabra.fi/KostiKaa/team-network-training/uploads/d6922ff6a34dd7c40fa504afcf45bdb4/image.png)

Blokkeihin jakoa varten teimme funktion/moduulin, joka määrittää kuvassa näkyvät koordinaatit.
- `coodinates.py`

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_8777cd618167f1b73d82529f743215f0.png)

## 1.3 Datassa havaitut poikkeamat

### 1.3.1 Hyvältä näyttävä esimerkkiarkki

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_5afaa6f39e0e54c819d7e1051146ccf5.png)

### 1.3.2 Outlierit - ohi arkin osuva vikadata 

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_84ab2dcd917324f6e13f111efdabf51a.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_3229c7300aef62a177bd352b84f58305.png)

### 1.3.3 Puutteelliset tiheyssensorimerkinnät

Datasta löytyi myös arkkeja, joissa tiheyssensorin merkinnät olivat puutteelliset. Tehtäessä blokkeihin jakoa tämä tuotti NaN-arvoja tiheysarvojen laskennassa. Työ asian sopivaksi ratkaisuksi on yhä kesken.

#### 1.3.3.1 Kokonaan puuttuva oikea laita

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_a82dc09afebdd82adaaa51af77f0a225.png)

#### 1.3.3.2 Vajaa oikea laita

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_b6c49d3ef7b13799192135595bd87d3c.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_fa1b71c22fe192c2a81950e5b370049f.png)

#### 1.3.3.3 Vajaa oikea ja vasen laita

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_0fc50aa97ef43aa3621b7866825b8929.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_38d89bfbdf7e0a9e70f2f43c7ba7f189.png)

#### 1.3.3.4 Kokonaan puuttuvat tiheyssensorimerkinnät

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_448f8a5719213612a43eaca1732c468e.png)

### 1.4 Haaste itse blokkien jaon alkuperäisideassa

Blokkeihin jaossa alkuperäinen ajatus oli asettaa aina yksi arvo yksiselitteisesti yhteen blokkiin ja laskea sitten näistä arvoista joko keskiarvot (kosteus, lämpötila, tiheys) tai kokonaismäärät ja kokonaismitat (oksat, lahoumat ja kaikki muut viat). Vikojen osalta tällainen yksiselitteinen jako oli mahdollinen, koska vian sijainti määritetään painovoimakeskipisteen mukaan.

Sen sijaan kosteus- ja tiheyssensorien yksi lukema otetaan aina neljän koordinaattipisteen muodostamasta alueesta. Tämän hetken blokkeihin määrittelyssämme alue, joka jakaantuu useamman blokin kesken, ei päädy antamaan lukemaansa millekään blokille, vaan jää pois laskutoimituksista.

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_e7e2ab822770a9b65385356926a33b25.png)

# 2 Piirrematriisin muodostaminen

## 2.1 Yhdeksän blokin piirrematriisi

Yhdeksän blokin piirrematriisi on tällä hetkellä pääideamme. Piirrematriisiin otetaan märkäarkista seuraavat peruspiirteet:
- (Tiedoston nimi)
- Leveys
- Pituus
- Paksuus

Lisäksi jokaiselle yhdeksän blokkia sisältää nämä sarakkeet
- Blokin kosteuden keskiarvo
- Blokin lämpötilan keskiarvo
- Blokin tiheyden keskiarvo
- Blokin oksien määrä
- Blokin oksien leveyksien summa
- Blokin lahoumien määrä
- Blokin lahoumien leveyksien summa
- Blokin kaikkien muiden vikojen määrä
- Blokin kaikkien muiden vikojen leveyksien summa

Toistaiseksi kuiva-arkista ei oteta tietoja piirrematriisiin, vaikka esimerkiksi kosteustieto voisikin olla merkityksillinen neuroverkolle.

Tällä metodilla piirrematriisiin tulee 3 + 9 * 9 = 84 saraketta ja tiedoston nimi.

Funktio `featureMatrix()` muodostaa piirrematriisin ensin pyytämällä funktiolta `coordinates()` arkin blokkien jakoon tarvittavat koordinaatit, ja funktiolta `createColumns()` arkin oleellisen datan. Näiden tietojen perusteella `featureMatrix()` muodostaa piirrematriisiin tarvittavat sarakkeet ja laskee keskiarvot/määrät eri piirteille. Lopputulemana funktio palauttaa Pandasin dataframen, joka sisältää piirrematriisin yhden rivin - eli kaiken tiedon yhdestä viiluarkista.

Täyden piirrematriisin muodostamisen hoitaa `create_full_feature_and_target_matrix()`-funktio, jonka tehtävänä on kutsua `featureMatrix()`-funktiota käyden kaiken datan läpi ja koostamalla niistä täyden piirrematriisin.

In [1]:
from Modules.full_feature_matrix import create_full_feature_and_target_matrix

In [2]:
%%time
X, Y = create_full_feature_and_target_matrix(printProgress=True)

CPU times: user 56 s, sys: 3.15 s, total: 59.1 s
Wall time: 54.8 s


In [13]:
X

,peelFile,m_uWidth,m_uLength,m_dThickness,B1MoistureAvg,B1TemperatureAvg,B1DensityAvg,B1KnotWidthSum,B1KnotCount,B1DecayWidthSum,...,B8AllOtherDefectCount,B9MoistureAvg,B9TemperatureAvg,B9DensityAvg,B9KnotWidthSum,B9KnotCount,B9DecayWidthSum,B9DecayCount,B9AllOtherDefectWidthSum,B9AllOtherDefectCount
0,20210505123334_85.datx,1750.23265,1599.76152,1.56,117.582404,33.010073,1004.081748,125.39330,11,26.9537,...,0,125.324911,33.845573,1028.419309,157.62055,19,39.25865,2,16.99255,2
1,20210505135613_40.datx,1750.23265,1603.25064,1.56,92.388576,30.617250,NaN,0.00000,0,0.0000,...,0,98.377927,27.752846,1345.817186,26.36775,4,0.00000,0,0.00000,0
2,20210505150804_9.datx,1750.23265,1599.18000,1.56,101.473622,34.158417,961.639685,0.00000,0,0.0000,...,0,102.674582,34.486729,NaN,12.89090,1,0.00000,0,7.03140,1
3,20210505125941_29.datx,1750.23265,1601.50608,1.56,88.373384,35.438937,827.377599,21.68015,2,0.0000,...,0,92.668053,35.587054,NaN,49.21980,3,0.00000,0,16.40660,4
4,20210505123435_23.datx,1750.23265,1600.34304,1.56,120.877675,35.854021,956.372132,22.26610,3,0.0000,...,1,115.523224,35.723542,960.543745,15.82065,3,0.00000,0,450.00960,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,20210505125748_61.datx,1750.23265,1600.92456,1.56,93.128344,27.922545,865.955931,65.04045,10,0.0000,...,0,96.907931,27.696730,884.802702,87.30655,9,68.55615,2,0.00000,0
142,20210505145008_84.datx,1750.23265,1601.50608,1.56,95.698248,36.123000,952.906867,56.25120,7,0.0000,...,0,99.479994,35.438239,947.239349,67.97020,9,0.00000,0,0.00000,0
143,20210505125913_15.datx,1614.87820,1596.85392,1.56,89.492242,35.878303,853.189124,43.94625,7,0.0000,...,0,87.113549,35.705853,NaN,60.93880,11,0.00000,0,0.00000,0
144,20210505124918_24.datx,1750.23265,1601.50608,1.56,89.761967,35.592812,912.424399,96.68175,11,0.0000,...,0,90.379964,34.953594,NaN,79.10325,9,94.92390,7,0.00000,0


In [10]:
Y

,dryFile,dryWidth
0,20210505172657_52.datx,1633.089727
1,20210505163725_33.datx,1600.206205
2,20210505155509_66.datx,1609.685322
3,20210505172211_90.datx,1614.120061
4,20210505172314_13.datx,1620.398997
...,...,...
141,20210505172725_63.datx,1628.156770
142,20210505163636_13.datx,1602.033861
143,20210505172304_9.datx,1633.337841
144,20210505164707_82.datx,1594.211502


## 2.2 Karvalakkipiirrematriisi

Koska yhdeksän blokin piirrematriisi sisältää niin paljon sarakkeita ja dataa on toisteksi vähän, ajattelimme, että testaamme luoda myös vahvasti karsitun version piirrematriisista. Tämä on vielä työn alla, mutta ajatus on, että karvalakkimalli olisi seuraavanlainen:

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_d28541b8abb248c4225ec7564ec0f71f.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_1aeebc4ed26918aadee0c4b25a63f143.png)

# 3 Tietokanta

Bull bladelle pystytettiin MariaDB-tietokanta Docker-konttiin. Kontti nostetaan pystyyn Dockerfilen avulla, ja idea on, että pystytyksen yhteydessä populoidaan varmistetut parit sinne samalla.

Lisäksi piirrematriisin muodostusfunktioon on työn alla mahdollisuus puskea data tietokantaan. Mahdollinen jako testi-, opetutus- ja validointisetteihin on vielä mietinnässä.

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_aa601fc1ee73f8bcf509bee1db29bb9e.png)

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_95d8fc5f11617bc85ef02de60d9e7507.png)

# 4 Neuroverkko

Neuroverkkoa lähdettiin työstämään TensorFlowlla. Ensiksi teimme hieman tutkimustyötä, että mistä ehkä kannattaisi lähteä liikkeelle ja miten asioita tehdään TensorFlowlla. Sitten otimme tavoitteeksi saada ensimmäisen version neuroverkosta laadittua, ajettua datan sen läpi ilman erroreita ja tulostettua virhefunktion arvon sekä tarkkuusarvon näkyviin.

**Neuroverkon simppeli prototyyppi**

```python
from keras.models import Sequential
from keras.layers import Dense

optim = tf.keras.optimizers.Adam(0.001) # (clipnorm=1.) was said to work when the gradient could be exploding

def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(n_outputs))

model.compile(loss='mae', optimizer=optim, metrics='mse')
    
EPOCHS=500
history = model.fit(train_set_norm, y_train, 
          epochs=EPOCHS,
          verbose=(1 if EPOCHS % 10 == 0 else 0),
          validation_data=(test_set_norm, y_test))
```

**Opetus**

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_6466059d29f14aac184fc3cea3d3a90d.png)
![image](https://gitlab.dclabra.fi/wiki/uploads/upload_a24bdd11289dcba4e698ff8710fac9b3.png)

**Itse ennusteet**

![image](https://gitlab.dclabra.fi/wiki/uploads/upload_76d488d22e0aad8b75031a52e6b502a9.png)

Ennusteet ovat siis vielä hyvin kaukana todellisesta, mutta dataakin on hyvin vähän. Virhefunktion arvo selvästi laskee ja ennusteet pyörivät samalla skaalalla, mutta menevät kuitenkin huti. Riittääkö data - ehtiikö neuroverkko oppia? Toisaalta seuraavana testikohteena on, että mitä tapahtuisi sillä karvalakkimallilla - pääsisikö neuroverkko ainakin lähemmäksi totuutta?